In [1]:
import os
import random
from copy import deepcopy

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import torch
import torch_geometric

import network
from convert import parse_dump
from lammps_scripts import LJSimulationStandalone
from simulation import (
    CompressionSimulation,
    LJSimulation,
    TemperatureRange,
    gen_sim_data,
    run_lammps_calc,
)
from utils import (
    add_edges_LJ,
    draw_graph,
    draw_network,
    inject_noise,
    randomize_LJ,
    recalc_bond,
    recalc_bonds,
)



In [2]:
def calc_p_ratio_box(simulation: list, index: int = -1) -> float:
    """Calculates Poisson ratio from the box data

    Parameters
    ----------
    simulation : list[Data]
        list of torch_geometric `Data` objects
    Returns
    -------
    float
        Poisson ratio
    """
    return -(simulation[index].box.y - simulation[0].box.y) / (
        simulation[index].box.x - simulation[0].box.x
    )

## LAMMPS strain speed

In [ ]:
main_dir = "/home/sergey/work/simulator_data_gen/lammps_strain_speed_dePablo"
os.makedirs(main_dir)
# step_size = 0.01
# lj_sim = LJSimulation(
#     n_atoms=100,
#     n_atom_types=4,
#     atom_sizes=[0.8, 1.0, 1.2, 1.4],
#     box_dim=[-40, 40, -40, 40, -1, 1],
#     temperature_range=TemperatureRange(0.0005, 0.0001, 10.0),
#     n_steps=30_000,
# )
# lj_sim.write_to_file(main_dir)
# run_lammps_calc(main_dir, input_file='lammps.in')

net = network.Network.from_data_file(input_file="dePablo_network.lmp", include_default_masses=1e6)
for bond in net.bonds:
    bond.bond_coefficient = 1/bond.length
net.set_angle_coeff(0.00)
net.write_to_file(os.path.join(main_dir, "network.lmp"))


sims = {}
for rate in (1e-6, 1e-5, 1e-4, 1e-3):
    subdir = os.path.join(main_dir, f'{float(rate)}')
    os.makedirs(subdir)
    net.write_to_file(os.path.join(subdir, "network.lmp"))

    comp_sim = CompressionSimulation(
        strain_direction='x',
        box_size=net.box.x,
        network_filename='network.lmp',
        dt=0.01,
        strain=0.01,
        strain_rate=rate,
        desired_step_size=0.001,
        random_seed=True,
        temperature_range=TemperatureRange(1e-7, 1e-7, 10.0)
    )
    comp_sim.write_to_file(subdir)
    run_lammps_calc(subdir, "in.deformation")

    sims[rate] = parse_dump(
        os.path.join(subdir, "dump.lammpstrj"),
        net,
        node_features='coord'
    )

In [ ]:
sims.keys()

In [ ]:
for size, sim in sims.items():
    if sim:
        try:
            plt.scatter(size, calc_p_ratio_box(sim), label=f'Strain rate: {size:.1e}')
        except ZeroDivisionError:
            print(f"Size {size} too much")
            continue

plt.xticks(list(sims.keys()))
plt.axvline(1e-5, linestyle='--', color='red', linewidth=1.0, label='default strain rate')
plt.xscale('log')
plt.xlabel("Strain rate")
plt.ylabel("$\\nu$")
plt.legend()
plt.show()

## LAMMPS High T Random seed

In [ ]:
main_dir = "/home/sergey/work/simulator_data_gen/lammps_random_seed"
step_size = 0.01
lj_sim = LJSimulation(
    n_atoms=100,
    n_atom_types=4,
    atom_sizes=[0.8, 1.0, 1.2, 1.4],
    box_dim=[-40, 40, -40, 40, -1, 1],
    temperature_range=TemperatureRange(0.0005, 0.0001, 10.0),
    n_steps=30_000,
)
lj_sim.write_to_file(main_dir)
run_lammps_calc(main_dir, input_file='lammps.in')

net = network.Network.from_atoms(input_file=os.path.join(main_dir, "coord.dat"), include_default_masses=1e6)
for bond in net.bonds:
    bond.bond_coefficient = 1/bond.length
net.set_angle_coeff(0.00)
net.write_to_file(os.path.join(main_dir, "network.lmp"))


for i in range(5):
    subdir = os.path.join(main_dir, f'{i+1}')
    os.makedirs(subdir)
    net.write_to_file(os.path.join(subdir, "network.lmp"))

    comp_sim = CompressionSimulation(
        strain_direction='x',
        box_size=net.box.x,
        network_filename='network.lmp',
        dt=step_size,
        strain=0.03,
        strain_rate=1e-5,
        random_seed=True,
        temperature_range=TemperatureRange(1e-5, 1e-5, 10.0)
    )
    comp_sim.write_to_file(subdir)
    run_lammps_calc(subdir, "in.deformation")

In [ ]:
from convert import parse_dump

random_sims = []
for i in range(5):
    subdir = os.path.join(main_dir, f'{i+1}')
    random_sims.append(parse_dump(
        os.path.join(subdir, "dump.lammpstrj"),
        net,
        node_features='coord',
        skip=1
        )
    )

In [ ]:
torch.save(random_sims, "/home/sergey/work/gnn/NN_Simulator/data/random_sims.pt")

## LAMMPS check T vs P

In [ ]:
import pickle

main_dir = "/home/sergey/work/simulator_data_gen/lammps_t_over_p"
data_dir = "/home/sergey/work/simulator_data_gen/one_over_l"
steps_to_check = (1, 15, 30, 45)
ts_to_check = (1e-7, 1e-3)
paths = []
for size_dir in os.listdir(data_dir):
    if size_dir != "data_generation.log":
        for network_dir in os.listdir(os.path.join(data_dir, size_dir)):
            for step in steps_to_check:
                path = os.path.join(data_dir, size_dir, network_dir, f"network{step}.lmp")
                paths.append(path)

for i in range(50):
    path = random.sample(paths, 1)[0]
    print(f"Step {i+1}: {path}")
    net = network.Network.from_data_file(
        input_file=path,
        include_default_masses=1e6,
        include_angles=True,
        include_dihedrals=False
    )
    for bond in net.bonds:
        bond.bond_coefficient = 1/bond.length
    net.set_angle_coeff(0.00)

    for t in ts_to_check:
        comp_path = os.path.join(main_dir, str(i+1), str(t))
        os.makedirs(comp_path)
        net.write_to_file(os.path.join(comp_path, "network.lmp"))
        comp_sim = CompressionSimulation(
            strain_direction='x',
            box_size=net.box.x,
            network_filename='network.lmp',
            dt=0.01,
            strain=0.01,
            strain_rate=1e-5,
            random_seed=True,
            temperature_range=TemperatureRange(t, t, 10.0)
        )
        comp_sim.write_to_file(comp_path)
        run_lammps_calc(comp_path, "in.deformation")


In [ ]:
data_lowT = []
data_highT = []

for i in range(50):
    sim_lowT = parse_dump(
        f"/home/sergey/work/simulator_data_gen/lammps_t_over_p/{i+1}/{1e-3}/dump.lammpstrj",
        network.Network.from_data_file(f"/home/sergey/work/simulator_data_gen/lammps_t_over_p/{i+1}/{1e-3}/network.lmp", include_default_masses=1e6),
        node_features='coord'
    )
    data_lowT.append(sim_lowT)
    sim_highT = parse_dump(
        f"/home/sergey/work/simulator_data_gen/lammps_t_over_p/{i+1}/{1e-7}/dump.lammpstrj",
        network.Network.from_data_file(f"/home/sergey/work/simulator_data_gen/lammps_t_over_p/{i+1}/{1e-7}/network.lmp", include_default_masses=1e6),
        node_features='coord'
    )
    data_highT.append(sim_highT)


In [ ]:
ps_high = [calc_p_ratio_box(sim) for sim in data_highT]
ps_low = [calc_p_ratio_box(sim) for sim in data_lowT]

In [ ]:
plt.scatter(ps_high, ps_low)
plt.xlabel("$T=10^{-3}$")
plt.ylabel("$T=10^{-7}$")
plt.title("P ratio vs Temperature")
plt.show()

## LAMMPS big step

In [ ]:
main_dir = "/home/sergey/work/simulator_data_gen/lammps_big_step"
os.makedirs(main_dir)
# step_size = 0.5
# lj_sim = LJSimulation(
#     n_atoms=150,
#     n_atom_types=4,
#     atom_sizes=[0.8, 1.0, 1.2, 1.4],
#     box_dim=[-40, 40, -40, 40, -1, 1],
#     temperature_range=TemperatureRange(0.0005, 0.0001, 10.0),
#     n_steps=30_000,
# )
# lj_sim.write_to_file(main_dir)
# run_lammps_calc(main_dir, input_file='lammps.in')

net = network.Network.from_data_file(input_file="/home/sergey/work/simulator_data_gen/dePablo_network.lmp", include_default_masses=1e6)
# net = network.Network.from_atoms(input_file=os.path.join(main_dir, "coord.dat"), include_default_masses=1e6)
net.set_angle_coeff(0.00)
net.write_to_file(os.path.join(main_dir, "network.lmp"))

sims = {}
for step_size in [0.005, 0.01, 0.02, 0.04, 0.08, 0.16]:
    subdir = os.path.join(main_dir, f'{float(step_size)}')
    os.makedirs(subdir, exist_ok=True)
    net.write_to_file(os.path.join(subdir, 'network.lmp'))
    comp_sim = CompressionSimulation(
        strain_direction='x',
        box_size=net.box.x,
        network_filename='network.lmp',
        dt=step_size,
        strain=0.03,
        strain_rate=1e-5,
        temperature_range=TemperatureRange(1e-7, 1e-7, 100)
    )
    comp_sim.write_to_file(subdir)
    run_lammps_calc(subdir, "in.deformation")
    sims[step_size] = parse_dump(
            os.path.join(subdir, "dump.lammpstrj"),
            net,
            node_features='coord'
        )

In [ ]:
def calc_p_ratio_box(simulation: list, index: int = -1) -> float:
    """Calculates Poisson ratio from the box data

    Parameters
    ----------
    simulation : list[Data]
        list of torch_geometric `Data` objects
    Returns
    -------
    float
        Poisson ratio
    """
    return -(simulation[index].box.y - simulation[0].box.y) / (
        simulation[index].box.x - simulation[0].box.x
    )

In [ ]:
for size, sim in sims.items():
    if sim and size < 0.16:
        plt.scatter(size, calc_p_ratio_box(sim, 50), label=f'Step size: {size}')

plt.legend()
plt.xlabel("Step size")
plt.ylabel("$\\nu$")
plt.show()

In [ ]:


sim = parse_dump(os.path.join(main_dir, "dump.lammpstrj"), net, node_features='coord')
print(len(sim))

In [ ]:
draw_network(net, periodic_edges=False)

## Rest

In [ ]:
raw_data_path = "/home/sergey/work/simulator_data_gen/data/binary/data_dePablo_OOL_0.05strain"
count = 0
for size_dir in os.listdir(raw_data_path):
    if size_dir != "data_generation.log":
        for net_dir in os.listdir(os.path.join(raw_data_path, size_dir)):
            for comp_dir in os.listdir(os.path.join(raw_data_path, size_dir, net_dir)):
                if comp_dir.startswith("comp_"): #and 'one_over_l' in comp_dir:
                    count += 1

print(count)

In [12]:
name_suffix = "SR=1e-06"
path = f"/home/sergey/work/simulator_data_gen/data/binary/data_dePablo_OOL_{name_suffix}"
chunks = []
for chunk_name in os.listdir(path):
    chunk = torch.load(os.path.join(path, chunk_name), weights_only=False)
    chunks.append(chunk)

len(chunks)

425

In [13]:
torch.save(chunks, f"/home/sergey/work/gnn/NN_Simulator/data/dePablo_networks_OOL_{name_suffix}.pt")

In [ ]:
os.chdir("/home/sergey/work/simulator_data_gen")
data_1  = torch.load("validation_Ttimes1.pt", weights_only=False)
data_10  = torch.load("validation_Ttimes10.pt", weights_only=False)
data_100  = torch.load("validation_Ttimes100.pt", weights_only=False)
data_1000  = torch.load("validation_Ttimes1000.pt", weights_only=False)
data_10000  = torch.load("validation_Ttimes10000.pt", weights_only=False)

In [ ]:
from utils import visualize_graphs

k = 5
visualize_graphs(data_1[k][0], data_10[k][0], periodic_edges=False)

In [ ]:
calc_dir = "/home/sergey/work/simulator_data_gen/scaling" # work

for i in range(5):
    loc_dir = os.path.join(calc_dir, str(i))
    os.makedirs(loc_dir)
    lj_sim = LJSimulation(
        n_atoms=120,
        atom_sizes=[16, 14, 12, 10],
        n_atom_types=4,
        box_dim=[-300, 300, -300, 300, -1, 1],
        temperature_range=TemperatureRange(0.0005, 0.0001, 10.0),
        n_steps=30_000,
    )
    lj_sim.write_to_file(loc_dir)
    run_lammps_calc(loc_dir, input_file='lammps.in')


In [ ]:
skip = 1
data = []
for i in sizes:
    sim = []
    dump_file = os.path.join(calc_dir, str(i), "dump.lammpstrj")

    with open(dump_file, "r", encoding="utf8") as f:
            content = f.readlines()

    timesteps: list[int] = []
    for index, line in enumerate(content):
        if "ITEM: TIMESTEP" in line:
            timesteps.append(index)

    for i in range(0, len(timesteps) - 1, skip):
        timestep_data = content[timesteps[i] : timesteps[i + 1]]
        atoms = [list(map(lambda x: float(x), line.split()[1:3])) for line in timestep_data[9:]]
        graph = torch_geometric.data.Data(x=torch.tensor(atoms))
        graph.box = network.Box(-30, 30, -30, 30, -0.1, 0.1)
        graph = add_edges_LJ(graph, r=2.0)
        graph.edge_attr = torch.ones_like(graph.edge_attr)
        sim.append(graph)
    data.append(sim)
    # datasets[edge_radius] = data

In [ ]:
torch.save(data, "/home/sergey/work/simulator_data_gen/diff_size.pt")

In [ ]:
for key, value in datasets.items():
    torch.save(value, f"/home/sergey/work/simulator_data_gen/LJ_data_r{key}.pt")

In [ ]:
from convert import parse_dump

ord_data = []
for i in sizes:
    loc_dir = os.path.join(calc_dir, str(i))
    ord_network = network.Network.from_data_file(
        os.path.join(loc_dir, f"network{i}.lmp"),
        include_default_masses=1e6,
    )
    sim = parse_dump(
        os.path.join(loc_dir, "dump.lammpstrj"),
        ord_network,
        node_features="coord"
    )

    ord_data.append(sim)

ord_data

In [ ]:
torch.save(data, "/home/sergey/work/gnn/NN_Simulator/LJ_data.pt")

In [ ]:
def draw_network(
    net: network.Network,
    edges: bool = True,
    periodic_edges: bool = True,
    box: bool = False,
    node_color: str = "skyblue",
    standalone: bool = True,
    node_size: float = 20,
    figure_scale: float = 3,
):
    if standalone:
        plt.figure(figsize=(8, 8))
    G = nx.Graph()
    # Add nodes
    for atom in net.atoms:
        G.add_node(atom.atom_id)

    pos = {atom.atom_id: (float(atom.y), float(atom.x)) for atom in net.atoms}

    # Add edges
    if edges:
        edge_index = [(bond.atom1.atom_id, bond.atom2.atom_id) for bond in net.bonds]
        naive_edge_lengths = [bond.atom1.dist(bond.atom2) for bond in net.bonds]
        for edge, length in zip(edge_index, naive_edge_lengths):
            if periodic_edges:
                G.add_edge(edge[0], edge[1])
            else:
                if length < net.box.x // 2:
                    G.add_edge(edge[0], edge[1])

    if box:
        B = nx.Graph()
        box_corners = [
            (net.box.x1, net.box.y1),
            (net.box.x1, net.box.y2),
            (net.box.x2, net.box.y1),
            (net.box.x2, net.box.y2),
        ]
        box_edges = [
            [box_corners[0], box_corners[1]],
            [box_corners[1], box_corners[2]],
            [box_corners[2], box_corners[3]],
            [box_corners[3], box_corners[0]],
        ]

        for corner, edge in zip(box_corners, box_edges):
            B.add_node(corner)
            B.add_edge(edge[0], edge[1])
        b_pos = {}
        b_pos[box_corners[0]] = (net.box.x1, net.box.y2)
        b_pos[box_corners[1]] = (net.box.x2, net.box.y2)
        b_pos[box_corners[2]] = (net.box.x2, net.box.y1)
        b_pos[box_corners[3]] = (net.box.x1, net.box.y1)

        nx.draw_networkx(B, b_pos, with_labels=False, node_color="black", node_size=1)

    nx.draw_networkx(
        G,
        pos,
        with_labels=False,
        node_color=node_color,
        node_size=node_size,
        font_size=10,
    )
    # nx.draw_networkx(G, with_labels=False, node_color=node_color, node_size=node_size, font_size=10)
    if standalone:
        plt.show()


In [ ]:
example = network.Network.from_data_file("data/raw/noised_prund/140_4/network_data/2/network.lmp")
draw_network(example, periodic_edges=False)

In [ ]:
from utils import load_data

data = load_data('noisy_networks.pt')